In [1]:
import cv2
import mediapipe as mp
from ultralytics import YOLO

In [6]:
# YOLO 모델 로드
yolo_model = YOLO('D:\\project\\prjvenv\\runs\\detect\\human_fall_s30\\weights\\best.pt')
print("YOLO 모델 로드 완료")

# MediaPipe 설정
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=False, min_detection_confidence=0.3)
mp_drawing = mp.solutions.drawing_utils
print("MediaPipe 설정 완료")

# 원하는 랜드마크 정의
DESIRED_LANDMARKS = [0, 11, 12, 15, 16, 23, 24, 25, 26, 27, 28]

# 동영상 파일 열기
video_path = r'D:\041.낙상사고 위험동작 영상-센서 쌍 데이터\3.개방데이터\1.데이터\Training\01.원천데이터\TS\영상\Y\SY\01880_Y_A_SY_C7\01880_Y_A_SY_C7.mp4'
cap = cv2.VideoCapture(video_path)
if not cap.isOpened():
    print("Error: 비디오 파일을 열 수 없습니다.")
    exit()
print("비디오 파일 열기 성공")

# 결과 동영상 저장을 위한 설정
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter('output_combined_3_수동bbox.mp4', fourcc, 60.0, (int(cap.get(3)), int(cap.get(4))))

frame_count = 0
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        print("비디오 종료")
        break

    frame_count += 1
    print(f"프레임 {frame_count} 처리 중")

    # YOLO로 사람 감지
    results = yolo_model(frame)
    print(f"YOLO 감지 결과: {len(results[0].boxes)} 개체 감지됨")

    for r in results:
        boxes = r.boxes
        for box in boxes:
            if int(box.cls) == 0:  # 사람 클래스
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                print(f"사람 감지: 바운딩 박스 좌표 ({x1}, {y1}, {x2}, {y2})")
                
                # 바운딩 박스 추출
                person_image = frame[y1:y2, x1:x2]
                
                # MediaPipe로 포즈 추정
                results_pose = pose.process(cv2.cvtColor(person_image, cv2.COLOR_BGR2RGB))
                
                if results_pose.pose_landmarks:
                    print("포즈 랜드마크 감지됨")
                    for idx, landmark in enumerate(results_pose.pose_landmarks.landmark):
                        if idx in DESIRED_LANDMARKS:
                            cx = int(landmark.x * person_image.shape[1])
                            cy = int(landmark.y * person_image.shape[0])
                            global_x = x1 + cx
                            global_y = y1 + cy
                            cv2.circle(frame, (global_x, global_y), 5, (0, 255, 0), -1)
                            print(f"랜드마크 {idx}: ({global_x}, {global_y})")
                else:
                    print("포즈 랜드마크를 감지하지 못함")
                
                # 바운딩 박스 그리기
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 0), 5)

    # 결과 표시
    cv2.namedWindow("YOLO + MediaPipe", cv2.WINDOW_NORMAL)
    cv2.resizeWindow("YOLO + MediaPipe", 1920, 1080)
    cv2.imshow("YOLO + MediaPipe", frame)
    
    # 결과 동영상 저장
    out.write(frame)

    # 'q' 키를 누르면 종료
    if cv2.waitKey(1) & 0xFF == ord('q'):
        print("사용자에 의해 종료됨")
        break

cap.release()
#out.release()
cv2.destroyAllWindows()
print("프로그램 종료")

YOLO 모델 로드 완료
MediaPipe 설정 완료
비디오 파일 열기 성공
프레임 1 처리 중

0: 384x640 1 Non_Fall, 15.6ms
Speed: 2.6ms preprocess, 15.6ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)
YOLO 감지 결과: 1 개체 감지됨
사람 감지: 바운딩 박스 좌표 (1536, 788, 1857, 1452)
포즈 랜드마크 감지됨
랜드마크 0: (1711, 902)
랜드마크 11: (1795, 940)
랜드마크 12: (1620, 936)
랜드마크 15: (1830, 1137)
랜드마크 16: (1564, 1109)
랜드마크 23: (1756, 1139)
랜드마크 24: (1657, 1139)
랜드마크 25: (1766, 1204)
랜드마크 26: (1608, 1216)
랜드마크 27: (1757, 1398)
랜드마크 28: (1605, 1407)
프레임 2 처리 중

0: 384x640 1 Non_Fall, 7.2ms
Speed: 0.0ms preprocess, 7.2ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)
YOLO 감지 결과: 1 개체 감지됨
사람 감지: 바운딩 박스 좌표 (1538, 788, 1857, 1452)
포즈 랜드마크 감지됨
랜드마크 0: (1713, 902)
랜드마크 11: (1795, 940)
랜드마크 12: (1622, 936)
랜드마크 15: (1831, 1139)
랜드마크 16: (1566, 1110)
랜드마크 23: (1757, 1136)
랜드마크 24: (1658, 1135)
랜드마크 25: (1768, 1210)
랜드마크 26: (1610, 1224)
랜드마크 27: (1759, 1403)
랜드마크 28: (1607, 1408)
프레임 3 처리 중

0: 384x640 1 Non_Fall, 3.0ms
Speed: 1.0ms pre

In [5]:
# YOLO 모델 로드
yolo_model = YOLO('D:\\project\\prjvenv\\runs\\detect\\human_fall_s30\\weights\\best.pt')
print("YOLO 모델 로드 완료")

# MediaPipe 설정
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=False, min_detection_confidence=0.3)
mp_drawing = mp.solutions.drawing_utils
print("MediaPipe 설정 완료")

# 원하는 랜드마크 정의
DESIRED_LANDMARKS = [0, 11, 12, 15, 16, 23, 24, 25, 26, 27, 28]

# 동영상 파일 열기
video_path = r'D:\041.낙상사고 위험동작 영상-센서 쌍 데이터\3.개방데이터\1.데이터\Training\01.원천데이터\TS\영상\N\N\02153_H_A_N_C2\02153_H_A_N_C2.mp4'
cap = cv2.VideoCapture(video_path)
if not cap.isOpened():
    print("Error: 비디오 파일을 열 수 없습니다.")
    exit()
print("비디오 파일 열기 성공")

# 결과 동영상 저장을 위한 설정
output_path = 'output_combined_yolo_6.mp4'
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
fps = cap.get(cv2.CAP_PROP_FPS)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

frame_count = 0
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        print("비디오 종료")
        break

    frame_count += 1
    print(f"프레임 {frame_count} 처리 중")

    # YOLO로 사람 감지
    results = yolo_model(frame)
    print(f"YOLO 감지 결과: {len(results[0].boxes)} 개체 감지됨")

    # YOLO 결과 시각화
    annotated_frame = results[0].plot()

    for r in results:
        boxes = r.boxes
        for box in boxes:
            if int(box.cls) == 0:  # 사람 클래스
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                print(f"사람 감지: 바운딩 박스 좌표 ({x1}, {y1}, {x2}, {y2})")
                
                # 바운딩 박스 추출
                person_image = frame[y1:y2, x1:x2]
                
                # MediaPipe로 포즈 추정
                results_pose = pose.process(cv2.cvtColor(person_image, cv2.COLOR_BGR2RGB))
                
                if results_pose.pose_landmarks:
                    print("포즈 랜드마크 감지됨")
                    for idx, landmark in enumerate(results_pose.pose_landmarks.landmark):
                        if idx in DESIRED_LANDMARKS:
                            cx = int(landmark.x * person_image.shape[1])
                            cy = int(landmark.y * person_image.shape[0])
                            global_x = x1 + cx
                            global_y = y1 + cy
                            cv2.circle(annotated_frame, (global_x, global_y), 5, (0, 255, 0), -1)
                            print(f"랜드마크 {idx}: ({global_x}, {global_y})")
                else:
                    print("포즈 랜드마크를 감지하지 못함")
                    
    out.write(annotated_frame)

    # 결과 표시
    cv2.namedWindow("YOLO + MediaPipe", cv2.WINDOW_NORMAL)
    cv2.resizeWindow("YOLO + MediaPipe", 1920, 1080)
    cv2.imshow("YOLO + MediaPipe", annotated_frame)
    
    out.write(frame)
    
    # 'q' 키를 누르면 종료
    if cv2.waitKey(1) & 0xFF == ord('q'):
        print("사용자에 의해 종료됨")
        break

cap.release()
out.release()
cv2.destroyAllWindows()
print("프로그램 종료")

YOLO 모델 로드 완료
MediaPipe 설정 완료
비디오 파일 열기 성공
프레임 1 처리 중

0: 384x640 1 Non_Fall, 27.0ms
Speed: 2.0ms preprocess, 27.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)
YOLO 감지 결과: 1 개체 감지됨
사람 감지: 바운딩 박스 좌표 (2603, 468, 3085, 1417)
포즈 랜드마크 감지됨
랜드마크 0: (2918, 595)
랜드마크 11: (2953, 740)
랜드마크 12: (2965, 732)
랜드마크 15: (2833, 1072)
랜드마크 16: (2820, 1033)
랜드마크 23: (2836, 1057)
랜드마크 24: (2853, 1053)
랜드마크 25: (2767, 1227)
랜드마크 26: (2774, 1214)
랜드마크 27: (2723, 1366)
랜드마크 28: (2725, 1351)
프레임 2 처리 중

0: 384x640 1 Non_Fall, 4.0ms
Speed: 1.0ms preprocess, 4.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)
YOLO 감지 결과: 1 개체 감지됨
사람 감지: 바운딩 박스 좌표 (2603, 468, 3088, 1417)
포즈 랜드마크 감지됨
랜드마크 0: (2916, 595)
랜드마크 11: (2953, 742)
랜드마크 12: (2960, 729)
랜드마크 15: (2833, 1072)
랜드마크 16: (2837, 1028)
랜드마크 23: (2830, 1034)
랜드마크 24: (2842, 1039)
랜드마크 25: (2761, 1223)
랜드마크 26: (2774, 1204)
랜드마크 27: (2721, 1360)
랜드마크 28: (2730, 1348)
프레임 3 처리 중

0: 384x640 1 Non_Fall, 3.0ms
Speed: 1.0ms pre

In [ ]:
# MacOS

import cv2
import mediapipe as mp
from ultralytics import YOLO

# YOLO 모델 로드
yolo_model = YOLO('path/to/your/best.pt')  # MacOS에서의 경로로 수정해주세요
print("YOLO 모델 로드 완료")

# MediaPipe 설정
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=False, min_detection_confidence=0.3)
mp_drawing = mp.solutions.drawing_utils
print("MediaPipe 설정 완료")

# 원하는 랜드마크 정의
DESIRED_LANDMARKS = [0, 11, 12, 15, 16, 23, 24, 25, 26, 27, 28]

# 웹캠 열기
cap = cv2.VideoCapture(0)  # 0은 기본 웹캠을 의미합니다
if not cap.isOpened():
    print("Error: 웹캠을 열 수 없습니다.")
    exit()
print("웹캠 열기 성공")

# 결과 동영상 저장을 위한 설정
output_path = 'output_combined_yolo_webcam.mp4'
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
fps = 60  # 웹캠 프레임 레이트, 필요에 따라 조정 가능
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

frame_count = 0
while True:
    ret, frame = cap.read()
    if not ret:
        print("웹캠에서 프레임을 읽을 수 없습니다.")
        break

    frame_count += 1
    print(f"프레임 {frame_count} 처리 중")

    # YOLO로 사람 감지
    results = yolo_model(frame)
    print(f"YOLO 감지 결과: {len(results[0].boxes)} 개체 감지됨")

    # YOLO 결과 시각화
    annotated_frame = results[0].plot()

    for r in results:
        boxes = r.boxes
        for box in boxes:
            if int(box.cls) == 0:  # 사람 클래스
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                print(f"사람 감지: 바운딩 박스 좌표 ({x1}, {y1}, {x2}, {y2})")
                
                # 바운딩 박스 추출
                person_image = frame[y1:y2, x1:x2]
                
                # MediaPipe로 포즈 추정
                results_pose = pose.process(cv2.cvtColor(person_image, cv2.COLOR_BGR2RGB))
                
                if results_pose.pose_landmarks:
                    print("포즈 랜드마크 감지됨")
                    for idx, landmark in enumerate(results_pose.pose_landmarks.landmark):
                        if idx in DESIRED_LANDMARKS:
                            cx = int(landmark.x * person_image.shape[1])
                            cy = int(landmark.y * person_image.shape[0])
                            global_x = x1 + cx
                            global_y = y1 + cy
                            cv2.circle(annotated_frame, (global_x, global_y), 5, (0, 255, 0), -1)
                            print(f"랜드마크 {idx}: ({global_x}, {global_y})")
                else:
                    print("포즈 랜드마크를 감지하지 못함")
                    
    out.write(annotated_frame)

    # 결과 표시
    cv2.imshow("YOLO + MediaPipe", annotated_frame)
    
    # 'q' 키를 누르면 종료
    if cv2.waitKey(1) & 0xFF == ord('q'):
        print("사용자에 의해 종료됨")
        break

cap.release()
out.release()
cv2.destroyAllWindows()
print("프로그램 종료")